# Chapter 3: Functions

---

Functions are fundamental building blocks in Python programming. This chapter explores best practices for writing clear, maintainable, and effective functions.

## Item 19: Never Unpack More Than Three Return Values

### Multiple Return Values

Python allows functions to return multiple values using tuple unpacking. However, **returning more than three values becomes error-prone and hard to read**.

### Basic Multiple Return Example

In [ ]:
# Simple example showing tuple unpacking
def get_min_max(numbers):
    """Return minimum and maximum values"""
    return min(numbers), max(numbers)


lengths = [63, 73, 72, 60, 67, 66, 71, 61, 72, 70]

minimum, maximum = get_min_max(lengths)

print(f'Min: {minimum}, Max: {maximum}')

Min: 60, Max: 73


### How Multiple Returns Work

Multiple values are returned as a **tuple**, which is then unpacked:

In [ ]:
# Unpacking works the same way as multiple assignment
first, second = 1, 2
assert first == 1
assert second == 2


# Function return is equivalent
def my_function():
    return 1, 2


first, second = my_function()
assert first == 1
assert second == 2

print("✓ Basic unpacking works correctly")

### Using Starred Expressions for Multiple Returns

In [ ]:
# Catch-all unpacking with starred expressions
def get_avg_ratio(numbers):
    """Return ratios relative to average, sorted descending"""
    # Calculate the mean value of all numbers in the list
    average = sum(numbers) / len(numbers)

    # Create a new list where each value is normalized against the average
    # Values > 1.0 are above average, values < 1.0 are below average
    scaled = [x / average for x in numbers]

    # Sort the scaled ratios in descending order (largest to smallest)
    scaled.sort(reverse=True)

    return scaled


# Unpack the sorted ratios using starred expression:
# - 'longest' gets the first element (highest ratio)
# - 'shortest' gets the last element (lowest ratio)
# - '*middle' captures all remaining elements in between as a list
longest, *middle, shortest = get_avg_ratio(lengths)

# Display the highest ratio as a percentage, right-aligned in 4 characters
print(f'Longest: {longest:>4.0%}')

# Display the lowest ratio as a percentage, right-aligned in 4 characters
print(f'Shortest: {shortest:>4.0%}')

# Show how many middle values were captured by the starred expression
print(f'Middle values: {len(middle)} items')

Longest: 108%
Shortest:  89%
Middle values: 8 items


### The Problem: Too Many Return Values

When functions return many values, **two major problems** arise:

1. **Easy to swap arguments** (all numeric, hard to track)
2. **Long, unreadable unpacking statements**

In [3]:
# BAD: Too many return values
def get_stats(numbers):
    minimum = min(numbers)
    maximum = max(numbers)
    count = len(numbers)
    average = sum(numbers) / count

    sorted_numbers = sorted(numbers)
    middle = count // 2
    if count % 2 == 0:
        lower = sorted_numbers[middle - 1]
        upper = sorted_numbers[middle]
        median = (lower + upper) / 2
    else:
        median = sorted_numbers[middle]

    return minimum, maximum, average, median, count


# CORRECT unpacking
minimum, maximum, average, median, count = get_stats(lengths)
print(f'Min: {minimum}, Max: {maximum}')
print(f'Average: {average}, Median: {median}, Count {count}')

Min: 60, Max: 73
Average: 67.5, Median: 68.5, Count 10


In [ ]:
# WRONG: Easy to accidentally swap median and average!
# This is valid Python but has SWAPPED median and average
minimum, maximum, median, average, count = get_stats(lengths)
print(f'Average: {average}, Median: {median}')
print("⚠️ Values are swapped but no error was raised!")

### Readability Problems with Long Unpacking

In [ ]:
# All these formats hurt readability:

# Wrapped with backslash
minimum, maximum, average, median, count = \
    get_stats(lengths)

# Parentheses wrapping
(minimum, maximum, average,
 median, count) = get_stats(lengths)

# Awkward trailing parenthesis
(minimum, maximum, average, median, count
 ) = get_stats(lengths)

print("All work, but all are hard to read!")

### Solution: Use namedtuple or Lightweight Classes

**Best Practice**: Never unpack more than **three variables**. For more complex returns, use:
- `namedtuple`
- Lightweight class
- Dataclass

In [4]:
from collections import namedtuple

# GOOD: Use namedtuple for multiple values
Stats = namedtuple('Stats', ['minimum', 'maximum', 'average', 'median', 'count'])


def get_stats_better(numbers):
    minimum = min(numbers)
    maximum = max(numbers)
    count = len(numbers)
    average = sum(numbers) / count

    sorted_numbers = sorted(numbers)
    middle = count // 2
    if count % 2 == 0:
        lower = sorted_numbers[middle - 1]
        upper = sorted_numbers[middle]
        median = (lower + upper) / 2
    else:
        median = sorted_numbers[middle]

    return Stats(minimum, maximum, average, median, count)


# Now accessing values is clear and self-documenting
stats = get_stats_better(lengths)  # noqa: E305
print(f'Min: {stats.minimum}, Max: {stats.maximum}')
print(f'Average: {stats.average}, Median: {stats.median}')
print(f'Count: {stats.count}')

Min: 60, Max: 73
Average: 67.5, Median: 68.5
Count: 10


### Key Takeaways

| ✓ Good Practice | ✗ Bad Practice |
|----------------|----------------|
| Return 1-3 values max | Return 4+ values |
| Use namedtuple for complex returns | Rely on position for many values |
| Use starred expressions for catch-all | Mix too many return types |

---

## Item 20: Prefer Raising Exceptions to Returning None

### The Problem with Returning None

Returning `None` for special cases is **error-prone** because:
- `None` evaluates to `False` in conditionals
- Zero, empty string, empty list also evaluate to `False`
- Easy to accidentally misinterpret results

In [ ]:
# BAD: Returning None for errors
def careful_divide_bad(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

# Checking for None
x, y = 1, 0
result = careful_divide_bad(x, y)
if result is None:
    print('Invalid inputs')
else:
    print(f'Result: {result}')

### The Trap: Zero Returns

In [ ]:
# BUG: This looks like an error check, but it's wrong!
x, y = 0, 5
result = careful_divide_bad(x, y)

# WRONG: This triggers even though the division succeeded!
if not result:
    print('Invalid inputs')  # This runs! But shouldn't
else:
    print(f'Result: {result}')

### Approach 1: Return Status Tuple (Not Recommended)

In [ ]:
# MEDIOCRE: Return success status + result
def careful_divide_tuple(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None


# Forces caller to check status
success, result = careful_divide_tuple(x, y)
if not success:
    print('Invalid inputs')
else:
    print(f'Result: {result}')

In [ ]:
# PROBLEM: Callers can ignore the status using underscore
_, result = careful_divide_tuple(x, y)
if not result:
    print('Invalid inputs')  # Still runs incorrectly!

### Approach 2: Raise Exceptions (BEST PRACTICE)

In [ ]:
from typing import Union


# GOOD: Raise exceptions for special cases with type annotations
def careful_divide(a: float, b: float)  -> float:
    """
    Divide two numbers with error handling.

    Args:
        a: Numerator (float)
        b: Denominator (float)

    Returns:
        float: Result of division

    Raises:
        ValueError: If division by zero is attempted
    """
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')


# Caller handles exception explicitly
x: int = 5
y: int = 2

try:
    result: float = careful_divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print(f'Result is {result:.1f}')

Result is 2.5


In [ ]:
# Exception is raised for actual errors
try:
    result = careful_divide(5, 0)
except ValueError as e:
    print(f'Caught error: {e}')

### Using Type Annotations with Exceptions

In [ ]:
# BEST: Type annotations + docstring documenting exceptions
def careful_divide(a: float, b: float) -> float:
    """Divides a by b.
    Raises:
        ValueError: When the inputs cannot be divided.
    """
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

# Type annotations make it clear the function always returns float
# Never returns None


result = careful_divide(10, 2)
print(f'Result: {result}')

Result: 5.0


### Comparison: None vs Exceptions

| Approach | Pros | Cons |
|----------|------|------|
| **Return None** | Simple | Error-prone, confuses falsy values |
| **Return (status, value)** | Explicit status | Can be ignored, verbose |
| **Raise Exception** ✓ | Clear, forces handling | Slightly more verbose |

---

## Item 21: Know How Closures Interact with Variable Scope

### Understanding Closures

A **closure** is a function that refers to variables from the scope in which it was defined.

In [ ]:
from collections.abc import MutableSequence, Set
from typing import TypeVar, Literal

# Type variable for sortable elements
T = TypeVar('T')


def sort_priority(values: MutableSequence[T], group: Set[T]) -> None:
    """
    Sort values in-place, prioritizing elements that appear in the group set.
    
    Elements in 'group' are sorted first (maintaining their relative order),
    followed by elements not in 'group' (also maintaining their relative order).
    
    Args:
        values: Mutable sequence to sort in-place
        group: Set of elements that should be prioritized
        
    Returns:
        None (sorts in-place)
    """
    def helper(x: T) -> tuple[Literal[0, 1], T]:
        """
        Return sort key tuple: (priority_level, original_value)
        
        Args:
            x: Element to generate sort key for
            
        Returns:
            Tuple where first element is 0 (high priority) if x is in group,
            or 1 (low priority) if x is not in group. Second element is the
            original value for stable sorting within priority levels.
        """
        if x in group:
            return (0, x)  # High priority
        return (1, x)      # Low priority
    
    values.sort(key=helper)


# Example usage with type annotations
numbers: list[int] = [8, 3, 1, 2, 5, 4, 7, 6]
group: set[int] = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)  # Output: [2, 3, 5, 7, 1, 4, 6, 8]

### Why Closures Work

Three key Python features enable closures:

1. **Closures** can reference variables from enclosing scopes
2. **Functions are first-class objects** (can be passed around)
3. **Tuple comparison** works element-by-element

In [ ]:
# Demonstrating first-class functions
def make_multiplier(n):
    def multiply(x):
        return x * n  # 'n' comes from enclosing scope
    return multiply

times_three = make_multiplier(3)
print(f'3 × 5 = {times_three(5)}')
print(f'3 × 10 = {times_three(10)}')

### The Scoping Problem

Attempting to modify enclosing scope variables creates a **scoping bug**:

In [ ]:
# BUG: Trying to modify enclosing scope
def sort_priority2(numbers, group):
    found = False  # Outer scope
    def helper(x):
        if x in group:
            found = True  # BUG: Creates NEW local variable!
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
found = sort_priority2(numbers, group)
print('Found:', found)  # False - WRONG!
print(numbers)          # Sorted correctly though

### Variable Scope Resolution Order

Python resolves variable references in this order:

1. Current function's scope
2. Enclosing scopes
3. Global scope
4. Built-in scope

**Assignment** creates a local variable unless told otherwise!

In [ ]:
# Demonstrating scope resolution
try:
    foo = does_not_exist * 5
except NameError as e:
    print(f"NameError: {e}")

### Solution 1: Using `nonlocal`

In [ ]:
# GOOD: Use nonlocal to modify enclosing scope
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found  # Tells Python to look in enclosing scope
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
found = sort_priority3(numbers, group)
print('Found:', found)  # True - CORRECT!
print(numbers)

### Solution 2: Helper Class (For Complex State)

In [ ]:
# BETTER: Use a class for more complex state
class Sorter:
    def __init__(self, group):
        self.group = group
        self.found = False
    
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sorter = Sorter(group)
numbers.sort(key=sorter)

assert sorter.found is True
print('Found:', sorter.found)
print(numbers)

### When to Use Each Approach

| Approach | Best For |
|----------|----------|
| **Simple closures** | Reading enclosing variables |
| **nonlocal** | Simple state modification |
| **Helper class** ✓ | Complex state or multiple values |

---

## Item 22: Reduce Visual Noise with Variable Positional Arguments

### The Power of *args

Variable positional arguments (`*args`) make function calls cleaner by accepting any number of positional arguments.

In [ ]:
# WITHOUT *args - verbose and noisy
def log_old(message, values):
    # PROBLEM 1: MANDATORY PARAMETER FOR OPTIONAL DATA
    # The 'values' parameter is required, even when the caller has no values to log.
    # This forces callers to pass an empty list [], creating unnecessary boilerplate.
    # The function signature doesn't reflect that values are conceptually optional.
    
    if not values:
        # PROBLEM 2: RUNTIME CHECKING FOR DESIGN FLAW
        # We need this conditional because the API design forces callers to pass
        # empty containers. This is a code smell - we're compensating at runtime
        # for a poor compile-time API design decision.
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')


# PROBLEM 3: AWKWARD CALLER EXPERIENCE - MUST CONSTRUCT LIST
# Even with just two values, caller must wrap them in list syntax
log_old('My numbers are', [1, 2])
# ^^^ Brackets [] add visual noise and cognitive overhead

# PROBLEM 4: ANNOYING EMPTY CONTAINER REQUIREMENT
# When caller has no values, they still must pass empty list
log_old('Hi there', [])  # Annoying empty list!
# ^^^ The empty [] conveys no semantic information but is syntactically required
# This violates the principle: "required parameters should be necessary"

# PROBLEM 5: INCONSISTENT MENTAL MODEL
# Callers must think differently for two scenarios:
#   Scenario A: "I have values" → wrap in list
#   Scenario B: "I have no values" → pass empty list
# The API should have ONE mental model, not two context-dependent ones

# PROBLEM 6: TYPE SAFETY ISSUES (no type hints shown)
# Without type hints, unclear that 'values' expects an iterable
# Caller might mistakenly try: log_old('Numbers', 1, 2) → TypeError
# The signature doesn't communicate the container requirement

# PROBLEM 7: INFLEXIBLE FOR VARIABLE ARGUMENT COUNTS
# If caller has values in variables, they must manually collect them:
a, b, c, d, e = 1, 2, 3, 4, 5
log_old('Five values', [a, b, c, d, e])
# ^^^ Must manually construct list each time

# PROBLEM 8: SEMANTIC MISMATCH
# The function semantically accepts "a message and optionally some values"
# But syntactically requires "a message AND a list (possibly empty)"
# The signature doesn't match the conceptual model


# ============================================================================
# SOLUTION: Using *args
# ============================================================================

def log_new(message, *values):
    # SOLUTION BENEFITS:
    # 1. 'values' is now truly optional - callers omit it when not needed
    # 2. No runtime check needed for empty case - handled by natural tuple behavior
    # 3. Python automatically packs arguments into tuple
    # 4. Signature matches conceptual model: "message + optional values"
    
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')


# IMPROVED CALLER EXPERIENCE:
log_new('Hi there')                    # Clean! No empty container needed
log_new('My numbers are', 1, 2)        # Natural! No list construction
log_new('Many values', 1, 2, 3, 4, 5)  # Flexible! Any number of arguments

# CONSISTENT MENTAL MODEL:
# All calls follow same pattern: log_new(message, [optional values...])
# No context-dependent thinking required

# TYPE-ANNOTATED VERSION:
def log_typed(message: str, *values: int) -> None:
    """
    Log a message with optional integer values.
    
    Args:
        message: The message to log
        *values: Optional integer values to append to message
    """
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')


# COMPARISON SUMMARY:
# ==================
# log_old:  Requires explicit container, even when empty
#           → Forces caller to think about implementation detail
#
# log_new:  Accepts natural argument list, Python handles packing
#           → Caller thinks only about the values themselves

In [ ]:
# WITH *args - clean and elegant
def log(message, *values):
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')

log('My numbers are', 1, 2)
log('Hi there')  # Much better!

In [ ]:
# ============================================================================
# EASY CODING EXERCISE: Introduction to *args
# ============================================================================

"""
BEGINNER-FRIENDLY PROBLEMS:
---------------------------

1. add_numbers(*numbers)
   - Add all the numbers together
   - Example: add_numbers(1, 2, 3) → 6
   - Example: add_numbers(10, 20) → 30

2. count_arguments(*args)
   - Count how many arguments were passed
   - Example: count_arguments(1, 2, 3, 4) → 4
   - Example: count_arguments('a', 'b') → 2

3. make_sentence(*words)
   - Join all words with spaces to make a sentence
   - Example: make_sentence('Hello', 'world') → 'Hello world'
   - Example: make_sentence('I', 'love', 'Python') → 'I love Python'

4. multiply_all(*numbers)
   - Multiply all numbers together
   - Example: multiply_all(2, 3, 4) → 24
   - Example: multiply_all(5, 2) → 10
"""


# ============================================================================
# YOUR SOLUTIONS GO HERE
# ============================================================================


def add_numbers(*numbers):
    """Add all numbers together."""
    # TODO: Write your code here
    pass


def count_arguments(*args):
    """Count how many arguments were passed."""
    # TODO: Write your code here
    pass


def make_sentence(*words):
    """Join all words with spaces."""
    # TODO: Write your code here
    pass


def multiply_all(*numbers):
    """Multiply all numbers together."""
    # TODO: Write your code here
    pass


# ============================================================================
# TEST YOUR SOLUTIONS
# ============================================================================

def test_solutions():
    print("Testing your solutions...\n")
    
    # Test 1
    print("Test 1: add_numbers")
    result1 = add_numbers(1, 2, 3)
    print(f"add_numbers(1, 2, 3) = {result1}")
    print(f"Expected: 6")
    print()
    
    # Test 2
    print("Test 2: count_arguments")
    result2 = count_arguments(1, 2, 3, 4)
    print(f"count_arguments(1, 2, 3, 4) = {result2}")
    print(f"Expected: 4")
    print()
    
    # Test 3
    print("Test 3: make_sentence")
    result3 = make_sentence('Hello', 'world')
    print(f"make_sentence('Hello', 'world') = '{result3}'")
    print(f"Expected: 'Hello world'")
    print()
    
    # Test 4
    print("Test 4: multiply_all")
    result4 = multiply_all(2, 3, 4)
    print(f"multiply_all(2, 3, 4) = {result4}")
    print(f"Expected: 24")


if __name__ == "__main__":
    test_solutions()

In [ ]:
# ============================================================================
# EASY SOLUTION: Introduction to *args - With Simple Explanations
# ============================================================================


def add_numbers(*numbers):
    """Add all numbers together."""
    
    # STEP 1: Understand what *numbers does
    # When you call: add_numbers(1, 2, 3)
    # Python creates: numbers = (1, 2, 3)  <- This is a tuple!
    
    # STEP 2: Use sum() to add all numbers in the tuple
    total = sum(numbers)
    
    # STEP 3: Return the result
    return total


def count_arguments(*args):
    """Count how many arguments were passed."""
    
    # EXPLANATION:
    # *args puts all arguments into a tuple
    # len() tells us how many items are in the tuple
    
    # When you call: count_arguments('a', 'b', 'c')
    # Python creates: args = ('a', 'b', 'c')
    # len(args) returns: 3
    
    return len(args)


def make_sentence(*words):
    """Join all words with spaces."""
    
    # EXPLANATION:
    # *words puts all word arguments into a tuple
    # ' '.join(words) puts a space between each word
    
    # When you call: make_sentence('I', 'love', 'Python')
    # Python creates: words = ('I', 'love', 'Python')
    # ' '.join(words) creates: 'I love Python'
    
    sentence = ' '.join(words)
    return sentence


def multiply_all(*numbers):
    """Multiply all numbers together."""
    
    # EXPLANATION:
    # We start with result = 1
    # Then multiply each number one by one
    
    # When you call: multiply_all(2, 3, 4)
    # Python creates: numbers = (2, 3, 4)
    # Loop does: 1 * 2 = 2, then 2 * 3 = 6, then 6 * 4 = 24
    
    result = 1
    for num in numbers:
        result = result * num
    
    return result


# ============================================================================
# VISUAL EXAMPLES - See How *args Works
# ============================================================================

def show_examples():
    """Show what's happening behind the scenes."""
    
    print("=" * 50)
    print("HOW *args WORKS - Visual Examples")
    print("=" * 50)
    print()
    
    # Example 1: add_numbers
    print("Example 1: add_numbers(5, 10, 15)")
    print("What Python does:")
    print("  Step 1: Create tuple → numbers = (5, 10, 15)")
    print("  Step 2: Calculate sum → 5 + 10 + 15 = 30")
    print(f"  Result: {add_numbers(5, 10, 15)}")
    print()
    
    # Example 2: count_arguments
    print("Example 2: count_arguments('a', 'b', 'c', 'd')")
    print("What Python does:")
    print("  Step 1: Create tuple → args = ('a', 'b', 'c', 'd')")
    print("  Step 2: Count items → len(args) = 4")
    print(f"  Result: {count_arguments('a', 'b', 'c', 'd')}")
    print()
    
    # Example 3: make_sentence
    print("Example 3: make_sentence('Hello', 'my', 'friend')")
    print("What Python does:")
    print("  Step 1: Create tuple → words = ('Hello', 'my', 'friend')")
    print("  Step 2: Join with space → 'Hello my friend'")
    print(f"  Result: '{make_sentence('Hello', 'my', 'friend')}'")
    print()
    
    # Example 4: multiply_all
    print("Example 4: multiply_all(2, 5, 3)")
    print("What Python does:")
    print("  Step 1: Create tuple → numbers = (2, 5, 3)")
    print("  Step 2: Multiply → 1 * 2 = 2")
    print("  Step 3: Multiply → 2 * 5 = 10")
    print("  Step 4: Multiply → 10 * 3 = 30")
    print(f"  Result: {multiply_all(2, 5, 3)}")
    print()


# ============================================================================
# TEST ALL SOLUTIONS
# ============================================================================

def test_solutions():
    print("=" * 50)
    print("TESTING ALL SOLUTIONS")
    print("=" * 50)
    print()
    
    # Test 1
    print("Test 1: add_numbers")
    result1 = add_numbers(1, 2, 3)
    print(f"  add_numbers(1, 2, 3) = {result1}")
    print(f"  Expected: 6")
    print(f"  Status: {'✓ PASS' if result1 == 6 else '✗ FAIL'}")
    print()
    
    # Test 2
    print("Test 2: count_arguments")
    result2 = count_arguments(1, 2, 3, 4)
    print(f"  count_arguments(1, 2, 3, 4) = {result2}")
    print(f"  Expected: 4")
    print(f"  Status: {'✓ PASS' if result2 == 4 else '✗ FAIL'}")
    print()
    
    # Test 3
    print("Test 3: make_sentence")
    result3 = make_sentence('Hello', 'world')
    print(f"  make_sentence('Hello', 'world') = '{result3}'")
    print(f"  Expected: 'Hello world'")
    print(f"  Status: {'✓ PASS' if result3 == 'Hello world' else '✗ FAIL'}")
    print()
    
    # Test 4
    print("Test 4: multiply_all")
    result4 = multiply_all(2, 3, 4)
    print(f"  multiply_all(2, 3, 4) = {result4}")
    print(f"  Expected: 24")
    print(f"  Status: {'✓ PASS' if result4 == 24 else '✗ FAIL'}")
    print()
    
    print("=" * 50)
    print("ALL TESTS COMPLETED!")
    print("=" * 50)


# ============================================================================
# KEY POINTS TO REMEMBER
# ============================================================================

"""
🎯 SIMPLE RULES FOR *args:

1. The * means "collect all arguments"
   def my_func(*args):  ← The star collects everything

2. Inside the function, you get a TUPLE
   my_func(1, 2, 3)  → args becomes (1, 2, 3)

3. You can use any name (not just 'args')
   def add(*numbers):  ← 'numbers' works too!

4. The tuple can be empty
   my_func()  → args becomes ()

5. You can do normal tuple things:
   - len(args) → count items
   - for item in args: → loop through
   - sum(args) → add numbers
   - ' '.join(args) → join strings

WHY USE *args?

WITHOUT *args (BAD):
  def add(a, b, c):
      return a + b + c
  
  add(1, 2, 3)  ✓ Works
  add(1, 2, 3, 4)  ✗ ERROR! Too many arguments

WITH *args (GOOD):
  def add(*numbers):
      return sum(numbers)
  
  add(1, 2, 3)  ✓ Works
  add(1, 2, 3, 4)  ✓ Works
  add(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)  ✓ Works!

*args makes your function flexible!
"""


# Run everything
if __name__ == "__main__":
    show_examples()
    print()
    test_solutions()

### Using * Operator to Unpack Sequences

In [ ]:
# Unpack a list as positional arguments
favorites = [7, 33, 99]
log('Favorite colors', *favorites)

# Without *, would pass the list itself as single argument
log('Favorite colors', favorites)  # Different behavior

### Problem 1: Memory Consumption with Generators

In [ ]:
# WARNING: *args exhausts generators into memory
def my_generator():
    for i in range(10):
        yield i

def my_func(*args):
    print(args)

it = my_generator()
my_func(*it)  # Entire generator is consumed into tuple!

In [ ]:
# For large generators, this could cause memory problems
def large_generator():
    for i in range(1000000):
        yield i

# DON'T DO THIS with huge generators!
# my_func(*large_generator())  # Would consume ~8MB of memory

### Problem 2: Adding New Positional Arguments

In [ ]:
# Initial function
def log_v1(message, *values):
    if not values:
        print(f'{message}')
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')

log_v1('Favorites', 7, 33)
log_v1('Hi there')

In [ ]:
# Adding a new positional argument BREAKS old callers!
def log_v2(sequence, message, *values):
    if not values:
        print(f'{sequence} - {message}')
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{sequence} - {message}: {values_str}')

log_v2(1, 'Favorites', 7, 33)  # New with sequence - OK
log_v2(1, 'Hi there')           # New message only - OK
log_v2('Favorite numbers', 7, 33)  # Old usage - BREAKS!
# Now 'Favorite numbers' is used as sequence!

### Best Practices for *args

✓ **Use *args when:**
- Number of arguments is small and known
- Passing literals or variable names
- Improving readability

✗ **Avoid *args when:**
- Working with large generators
- Function interface might expand
- Arguments could be confused

**Solution**: Use keyword-only arguments for extensibility (see Item 25)

---

## Item 23: Provide Optional Behavior with Keyword Arguments

### Keyword Arguments Basics

In [ ]:
# Function with positional parameters
def remainder(number, divisor):
    return number % divisor

# All these calls are equivalent:
assert remainder(20, 7) == 6
assert remainder(20, divisor=7) == 6
assert remainder(number=20, divisor=7) == 6
assert remainder(divisor=7, number=20) == 6

print("✓ All calling styles produce same result")

In [ ]:
# Positional arguments must come before keyword arguments
try:
    remainder(number=20, 7)
except SyntaxError:
    print("SyntaxError: positional argument follows keyword argument")

In [ ]:
# Each argument can only be specified once
try:
    remainder(20, number=7)
except TypeError as e:
    print(f"TypeError: {e}")

### Using ** Operator with Dictionaries

In [ ]:
# Pass dictionary contents as keyword arguments
my_kwargs = {
    'number': 20,
    'divisor': 7,
}
assert remainder(**my_kwargs) == 6

# Mix with positional or keyword arguments
my_kwargs = {'divisor': 7}
assert remainder(number=20, **my_kwargs) == 6

# Multiple ** operators
my_kwargs = {'number': 20}
other_kwargs = {'divisor': 7}
assert remainder(**my_kwargs, **other_kwargs) == 6

### Catching All Keyword Arguments with **kwargs

In [ ]:
# Accept any keyword arguments
def print_parameters(**kwargs):
    for key, value in kwargs.items():
        print(f'{key} = {value}')

print_parameters(alpha=1.5, beta=9, gamma=4)

### Benefit 1: Clarity

In [ ]:
# Positional - unclear which is number, which is divisor
result = remainder(20, 7)

# Keyword - immediately obvious
result = remainder(number=20, divisor=7)
print(f"Result: {result}")

### Benefit 2: Default Values

In [ ]:
# Function for calculating flow rate
def flow_rate(weight_diff, time_diff, period=1):
    return (weight_diff / time_diff) * period

weight_diff = 0.5
time_diff = 3

# Default: per second
flow_per_second = flow_rate(weight_diff, time_diff)
print(f'{flow_per_second:.3} kg per second')

# Specify: per hour
flow_per_hour = flow_rate(weight_diff, time_diff, period=3600)
print(f'{flow_per_hour:.3} kg per hour')

### Benefit 3: Backward Compatibility

In [ ]:
# Extend function without breaking existing code
def flow_rate_extended(weight_diff, time_diff,
                       period=1, units_per_kg=1):
    return ((weight_diff * units_per_kg) / time_diff) * period

# Old callers still work
flow_per_second = flow_rate_extended(weight_diff, time_diff)
print(f'{flow_per_second:.3} kg per second')

# New callers can use new feature
pounds_per_hour = flow_rate_extended(
    weight_diff, time_diff,
    period=3600, units_per_kg=2.2
)
print(f'{pounds_per_hour:.3} pounds per hour')

### Warning: Positional Optional Arguments Can Be Confusing

In [ ]:
# CONFUSING: What do 3600 and 2.2 represent?
pounds_per_hour = flow_rate_extended(weight_diff, time_diff, 3600, 2.2)

# CLEAR: Use keywords for optional arguments
pounds_per_hour = flow_rate_extended(
    weight_diff, time_diff,
    period=3600, units_per_kg=2.2
)
print(f'{pounds_per_hour:.3} pounds per hour')

### Best Practices Summary

| Practice | Benefit |
|----------|----------|
| Use keyword arguments | Clarity at call site |
| Provide default values | Reduce repetition |
| Add optional keywords | Backward compatibility |
| Always use keywords for optional args | Avoid confusion |

---

## Item 24: Use None and Docstrings to Specify Dynamic Default Arguments

### The Problem: Mutable Default Arguments

In [ ]:
from datetime import datetime
from time import sleep

# BUG: Default argument evaluated only once at function definition!
def log_bad(message, when=datetime.now()):
    print(f'{when}: {message}')

log_bad('Hi there!')
sleep(0.1)
log_bad('Hello again!')  # Same timestamp!

### Why This Happens

Default argument values are evaluated **only once**, when the function is defined (at module load time).

After the module loads, `datetime.now()` is **never called again**!

### Solution: Use None with Docstring

In [ ]:
# GOOD: Use None as default, allocate in function body
def log(message, when=None):
    """Log a message with a timestamp.
    
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

log('Hi there!')
sleep(0.1)
log('Hello again!')  # Different timestamps now!

### The Mutable Default Trap

In [ ]:
import json

# DANGER: Mutable default argument
def decode_bad(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

# All calls share the SAME dictionary!
foo = decode_bad('bad data')
foo['stuff'] = 5

bar = decode_bad('also bad')
bar['meep'] = 1

print('Foo:', foo)
print('Bar:', bar)
print('They are the same object!', foo is bar)

### Correct Implementation

In [ ]:
# GOOD: Use None, create new dict in function
def decode(data, default=None):
    """Load JSON data from a string.
    
    Args:
        data: JSON data to decode.
        default: Value to return if decoding fails.
            Defaults to an empty dictionary.
    """
    try:
        return json.loads(data)
    except ValueError:
        if default is None:
            default = {}
        return default

# Now each call gets its own dictionary
foo = decode('bad data')
foo['stuff'] = 5

bar = decode('also bad')
bar['meep'] = 1

print('Foo:', foo)
print('Bar:', bar)
assert foo is not bar

### Using Type Annotations with None Defaults

In [ ]:
from typing import Optional

# Type annotations make None default explicit
def log_typed(message: str,
               when: Optional[datetime] = None) -> None:
    """Log a message with a timestamp.
    
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

log_typed('Type-annotated message')

### Common Dynamic Default Scenarios

| Default Value Type | Problem | Solution |
|-------------------|---------|----------|
| `datetime.now()` | Evaluated once at import | Use `None`, call in function |
| `{}` (empty dict) | Shared across calls | Use `None`, create `{}` in function |
| `[]` (empty list) | Shared across calls | Use `None`, create `[]` in function |
| Any mutable object | Shared state | Use `None`, create fresh object |

---

## Item 25: Enforce Clarity with Keyword-Only and Positional-Only Arguments

### The Problem: Ambiguous Arguments

In [ ]:
# Confusing Boolean arguments
def safe_division_bad(number, divisor,
                      ignore_overflow,
                      ignore_zero_division):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

# CONFUSING: Which Boolean is which?
result = safe_division_bad(1.0, 10**500, True, False)
print(result)

result = safe_division_bad(1.0, 0, False, True)
print(result)

### Improvement: Optional Keyword Arguments

In [ ]:
# Better: Default values
def safe_division_b(number, divisor,
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

# Much clearer with keywords
result = safe_division_b(1.0, 10**500, ignore_overflow=True)
print(result)

result = safe_division_b(1.0, 0, ignore_zero_division=True)
print(result)

In [ ]:
# Problem: Still can use positional arguments (confusing!)
result = safe_division_b(1.0, 10**500, True, False)
assert result == 0

### Solution: Keyword-Only Arguments (*)

In [ ]:
"""
Enhanced Safe Division Function with Keyword-Only Arguments

This module demonstrates Python's keyword-only parameter syntax, which enforces
explicit parameter naming to improve code clarity and prevent argument errors.
"""


def safe_division_c(number, divisor, *,
                    ignore_overflow=False,
                    ignore_zero_division=False):
    """
    Perform division with configurable exception handling.
    
    This function showcases the "keyword-only argument" pattern using the bare
    asterisk (*) separator in the parameter list.
    
    Parameters
    ----------
    number : float
        The numerator (dividend) in the division operation.
        Can be passed positionally or by keyword.
        
    divisor : float
        The denominator (divisor) in the division operation.
        Can be passed positionally or by keyword.
        
    * : separator
        CRITICAL DESIGN ELEMENT: The bare asterisk creates a boundary.
        All parameters AFTER this point MUST be passed using keyword syntax.
        This is not a parameter itself—it's a syntactic marker.
        
    ignore_overflow : bool, keyword-only, default=False
        Controls behavior when division results in numeric overflow.
        - If True: Returns 0 as a safe fallback value
        - If False: Allows OverflowError to propagate to caller
        MUST be passed as: ignore_overflow=True (not positionally)
        
    ignore_zero_division : bool, keyword-only, default=False
        Controls behavior when divisor is zero.
        - If True: Returns float('inf') following mathematical convention
        - If False: Allows ZeroDivisionError to propagate to caller
        MUST be passed as: ignore_zero_division=True (not positionally)
    
    Returns
    -------
    float
        The result of number / divisor, or a fallback value if exceptions
        are suppressed via the ignore_* parameters.
    
    Raises
    ------
    OverflowError
        If the division result exceeds float representation limits AND
        ignore_overflow=False (default behavior).
        
    ZeroDivisionError
        If divisor is zero AND ignore_zero_division=False (default behavior).
    
    Examples
    --------
    Valid usage with keyword-only arguments:
    >>> safe_division_c(10, 2)
    5.0
    
    >>> safe_division_c(1.0, 0, ignore_zero_division=True)
    inf
    
    >>> safe_division_c(1.0, 10**500, ignore_overflow=True)
    0
    
    Invalid usage (will raise TypeError):
    >>> safe_division_c(1.0, 10**500, True, False)
    TypeError: safe_division_c() takes 2 positional arguments but 4 were given
    
    Design Rationale
    ----------------
    The keyword-only pattern provides three key benefits:
    
    1. SEMANTIC CLARITY: Call sites become self-documenting
       - Compare: func(1, 2, True, False)  # What do these bools mean?
       - With:    func(1, 2, ignore_overflow=True, ignore_zero=False)
    
    2. API STABILITY: Can reorder or add keyword-only params without breaking code
       - Existing callers unaffected by parameter reordering
       - New parameters with defaults don't break backward compatibility
    
    3. ERROR PREVENTION: Impossible to accidentally swap boolean flags
       - Type checker and runtime both catch positional usage errors
    """
    
    try:
        # Attempt the division operation
        # This is the "happy path" where no exceptions occur
        return number / divisor
        
    except OverflowError:
        # OVERFLOW HANDLING BRANCH
        # Triggered when the result exceeds Python's float representation limits
        # Example: 1.0 / 10**-500 produces a number too large to represent
        
        if ignore_overflow:
            # SUPPRESSION MODE: Caller requested graceful handling
            # Return 0 as a safe, predictable fallback value
            # This allows computation to continue rather than crashing
            return 0
        else:
            # STRICT MODE: Caller wants to know about overflow conditions
            # Re-raise the exception so caller can handle it explicitly
            # This is the default behavior (ignore_overflow=False)
            raise
            
    except ZeroDivisionError:
        # ZERO DIVISION HANDLING BRANCH
        # Triggered when divisor is exactly zero
        # Mathematically undefined, but we offer configurable behavior
        
        if ignore_zero_division:
            # SUPPRESSION MODE: Return mathematical convention for infinity
            # float('inf') represents positive infinity in IEEE 754 standard
            # This matches mathematical limit behavior: lim(x/ε) as ε→0 = ∞
            return float('inf')
        else:
            # STRICT MODE: Caller wants explicit notification of division by zero
            # Re-raise the exception for explicit handling upstream
            # This is the default behavior (ignore_zero_division=False)
            raise


# ============================================================================
# DEMONSTRATION: Keyword-Only Argument Enforcement
# ============================================================================

if __name__ == "__main__":
    print("=" * 70)
    print("DEMONSTRATION: Keyword-Only Parameter Enforcement")
    print("=" * 70)
    
    # -------------------------------------------------------------------------
    # VALID USAGE EXAMPLES
    # -------------------------------------------------------------------------
    
    print("\n✓ VALID: Basic division (no exception handling)")
    result = safe_division_c(10, 2)
    print(f"  safe_division_c(10, 2) = {result}")
    
    print("\n✓ VALID: Using keyword arguments correctly")
    result = safe_division_c(1.0, 0, ignore_zero_division=True)
    print(f"  safe_division_c(1.0, 0, ignore_zero_division=True) = {result}")
    
    print("\n✓ VALID: Multiple keyword arguments")
    result = safe_division_c(1.0, 10**500, 
                            ignore_overflow=True, 
                            ignore_zero_division=False)
    print(f"  safe_division_c(1.0, 10**500, ignore_overflow=True, ...) = {result}")
    
    print("\n✓ VALID: Positional for number/divisor, keyword for flags")
    result = safe_division_c(100, 5, ignore_overflow=False)
    print(f"  safe_division_c(100, 5, ignore_overflow=False) = {result}")
    
    # -------------------------------------------------------------------------
    # INVALID USAGE EXAMPLE (DEMONSTRATES THE ERROR)
    # -------------------------------------------------------------------------
    
    print("\n" + "=" * 70)
    print("✗ INVALID: Attempting to pass keyword-only args positionally")
    print("=" * 70)
    
    try:
        # This call attempts to pass 4 positional arguments
        # But the function signature only accepts 2 positional args (number, divisor)
        # The bare * separator makes ignore_overflow and ignore_zero_division
        # keyword-only, so they CANNOT be passed positionally
        result = safe_division_c(1.0, 10**500, True, False)
        print(f"  Result: {result}")  # This line will never execute
        
    except TypeError as e:
        # Python's runtime catches the violation and raises TypeError
        print(f"\n  TypeError caught (as expected):")
        print(f"  {e}")
        print(f"\n  Explanation:")
        print(f"  - The function accepts 2 positional arguments: number, divisor")
        print(f"  - The bare * separator makes all following parameters keyword-only")
        print(f"  - Attempted to pass 4 positional arguments: (1.0, 10**500, True, False)")
        print(f"  - Python rejects this at the call site before any code executes")
    
    # -------------------------------------------------------------------------
    # CORRECT VERSION OF THE FAILED CALL
    # -------------------------------------------------------------------------
    
    print("\n" + "=" * 70)
    print("✓ CORRECTED: Same intent, proper keyword syntax")
    print("=" * 70)
    
    result = safe_division_c(1.0, 10**500, 
                            ignore_overflow=True, 
                            ignore_zero_division=False)
    print(f"  safe_division_c(1.0, 10**500,")
    print(f"                  ignore_overflow=True,")
    print(f"                  ignore_zero_division=False)")
    print(f"  Result: {result}")
    
    print("\n" + "=" * 70)
    print("KEY TAKEAWAY")
    print("=" * 70)
    print("""
The bare asterisk (*) in the function signature enforces an API contract:
- Parameters BEFORE the * can be passed positionally or by keyword
- Parameters AFTER the * MUST be passed by keyword
- This prevents bugs from positional argument confusion
- Makes code more readable and maintainable
- Allows API evolution without breaking existing callers
    """)

In [ ]:
# Keywords work perfectly
result = safe_division_c(1.0, 0, ignore_zero_division=True)
assert result == float('inf')
print("✓ Keyword-only arguments enforce clarity")

### Problem: Parameter Names as Interface

In [ ]:
# Callers can use parameter names
assert safe_division_c(number=2, divisor=5) == 0.4
assert safe_division_c(divisor=5, number=2) == 0.4
assert safe_division_c(2, divisor=5) == 0.4

In [ ]:
# Problem: Renaming parameters breaks code!
def safe_division_c_renamed(numerator, denominator, *,
                            ignore_overflow=False,
                            ignore_zero_division=False):
    try:
        return numerator / denominator
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

# Old code breaks!
try:
    safe_division_c_renamed(number=2, divisor=5)
except TypeError as e:
    print(f"TypeError: {e}")

### Solution: Positional-Only Arguments (/)

Python 3.8+ supports positional-only parameters using `/`

In [ ]:
# BEST: Positional-only parameters before /
def safe_division_d(numerator, denominator, /, *,
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        return numerator / denominator
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

# Positional arguments work
assert safe_division_d(2, 5) == 0.4
print("✓ Positional arguments work")

In [ ]:
# Keywords fail for positional-only parameters
try:
    safe_division_d(numerator=2, denominator=5)
except TypeError as e:
    print(f"TypeError: {e}")
    print("✓ Cannot use keywords for positional-only params")

### Flexible Parameters: Between / and *

In [ ]:
# Parameters between / and * can be passed either way
def safe_division_e(numerator, denominator, /,
                    ndigits=10, *,
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        fraction = numerator / denominator
        return round(fraction, ndigits)
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

# All these work!
result = safe_division_e(22, 7)
print(result)

result = safe_division_e(22, 7, 5)  # ndigits by position
print(result)

result = safe_division_e(22, 7, ndigits=2)  # ndigits by keyword
print(result)

### Parameter Type Summary

```python
def func(pos_only, /, pos_or_kwd, *, kwd_only):
    pass
```

| Position | Type | Symbol | Usage |
|----------|------|--------|-------|
| Before `/` | Positional-only | `/` | Must use position |
| Between `/` and `*` | Flexible | — | Position or keyword |
| After `*` | Keyword-only | `*` | Must use keyword |

---

## Item 26: Define Function Decorators with functools.wraps

### What Are Decorators?

Decorators allow you to **wrap functions** to modify their behavior before and after execution.

In [ ]:
# Simple decorator to trace function calls
# Decorators wrap functions to add behavior without modifying their source code
def trace(func):
    # The wrapper function intercepts calls to the decorated function
    # It receives all positional (*args) and keyword (**kwargs) arguments
    def wrapper(*args, **kwargs):
        # Execute the original function with the passed arguments
        # Store the result so we can log it and return it
        result = func(*args, **kwargs)
        
        # Print formatted trace showing function name, arguments, and return value
        # !r uses repr() for unambiguous output (shows quotes for strings, etc.)
        print(f'{func.__name__}({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        
        # Return the original function's result to maintain normal behavior
        # The caller gets the same value as if the function wasn't decorated
        return result
    
    # Return the wrapper function that will replace the original
    # This is what actually gets bound to the decorated function name
    return wrapper


# Apply decorator using @ syntax
# This is syntactic sugar for: fibonacci = trace(fibonacci)
@trace
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    
    # Base cases: fib(0)=0 and fib(1)=1
    # Without these, recursion would never terminate
    if n in (0, 1):
        return n
    
    # Recursive case: fib(n) = fib(n-2) + fib(n-1)
    # Each call triggers the trace decorator, showing the entire call tree
    return (fibonacci(n - 2) + fibonacci(n - 1))
# Regular comment - default comment color



# Test the decorated function
# This will show all recursive calls due to the trace decorator
result = fibonacci(4)

# Print the final result after all recursive calls complete
# The trace output above shows the execution tree
print(f'\nFinal result: {result}')

In [ ]:
"""
SIMPLE DECORATOR EXAMPLE
========================
A beginner-friendly introduction to Python decorators
"""

# ============================================================================
# EXAMPLE 1: The Simplest Possible Decorator
# ============================================================================

def make_loud(func):
    """
    A decorator that makes any function's output LOUD (uppercase)
    """
    # This wrapper function will replace the original function
    def wrapper():
        # Call the original function
        result = func()
        # Modify the result - make it LOUD!
        loud_result = result.upper()
        return loud_result
    
    # Return the wrapper (not calling it, just returning it)
    return wrapper


# Using the decorator with @ symbol
@make_loud
def greet():
    return "hello"


# Test it
print("Example 1: Simple decorator")
print(greet())  # Prints: HELLO
print()


# ============================================================================
# EXAMPLE 2: What's Really Happening (No @ Symbol)
# ============================================================================

def say_goodbye():
    return "goodbye"

# This is what @make_loud actually does behind the scenes:
say_goodbye = make_loud(say_goodbye)

print("Example 2: Same thing without @ symbol")
print(say_goodbye())  # Prints: GOODBYE
print()


# ============================================================================
# EXAMPLE 3: Decorator with Arguments
# ============================================================================

def add_excitement(func):
    """
    A decorator that adds exclamation marks to output
    """
    def wrapper(name):  # Now wrapper accepts arguments
        # Call original function with the argument
        result = func(name)
        # Add excitement!
        excited_result = result + "!!!"
        return excited_result
    return wrapper


@add_excitement
def welcome(name):
    return f"Welcome {name}"


print("Example 3: Decorator with function arguments")
print(welcome("Alice"))  # Prints: Welcome Alice!!!
print(welcome("Bob"))    # Prints: Welcome Bob!!!
print()


# ============================================================================
# EXAMPLE 4: Combining Multiple Decorators
# ============================================================================

def add_hearts(func):
    """
    A decorator that adds hearts around output
    """
    def wrapper(name):
        result = func(name)
        return f"❤️ {result} ❤️"
    return wrapper


# Stack decorators - they apply from bottom to top
@add_hearts
@add_excitement
def say_hello(name):
    return f"Hello {name}"


print("Example 4: Multiple decorators stacked")
print(say_hello("Charlie"))
# First: "Hello Charlie" 
# Then add_excitement: "Hello Charlie!!!"
# Then add_hearts: "❤️ Hello Charlie!!! ❤️"
print()


# ============================================================================
# EXAMPLE 5: Practical Use - Timer Decorator
# ============================================================================

import time

def timer(func):
    """
    A decorator that measures how long a function takes to run
    """
    def wrapper():
        # Record start time
        start = time.time()
        
        # Run the original function
        result = func()
        
        # Record end time
        end = time.time()
        
        # Calculate and print duration
        duration = end - start
        print(f"⏱️  {func.__name__} took {duration:.4f} seconds")
        
        return result
    return wrapper


@timer
def slow_function():
    """This function sleeps for 1 second"""
    time.sleep(1)
    return "Done!"


print("Example 5: Practical timer decorator")
result = slow_function()
print(f"Result: {result}")
print()


# ============================================================================
# VISUAL SUMMARY
# ============================================================================

print("=" * 60)
print("VISUAL SUMMARY: What @decorator Does")
print("=" * 60)
print("""
WITHOUT DECORATOR:
------------------
def greet():
    return "hello"

greet() ──────> "hello"


WITH DECORATOR:
---------------
@make_loud
def greet():
    return "hello"

greet() ──> [wrapper] ──> original greet() ──> "hello"
               │                                   │
               └─────────── .upper() ──────────────┘
                                │
                                v
                            "HELLO"

KEY CONCEPT:
- The decorator WRAPS the original function
- When you call greet(), you're actually calling the wrapper
- The wrapper can do stuff before and after calling the original
""")

In [ ]:
"""
SUPER SIMPLE DECORATOR - Arithmetic Example
============================================
One easy example to understand decorators
"""

# This is the decorator - it doubles any result
def double_result(func):
    """
    This decorator takes any number a function returns
    and multiplies it by 2
    """
    # wrapper is the new function that will replace the original
    def wrapper(a, b):
        # Call the original function
        result = func(a, b)
        
        # Double the result
        doubled = result * 2
        
        print(f"Original result: {result}")
        print(f"After doubling: {doubled}")
        
        return doubled
    
    return wrapper


# WITHOUT decorator - normal function
def add(a, b):
    return a + b

print("WITHOUT DECORATOR:")
print(f"add(3, 5) = {add(3, 5)}")
print()


# WITH decorator - same function but decorated
@double_result
def add_doubled(a, b):
    return a + b

print("WITH @double_result DECORATOR:")
result = add_doubled(3, 5)
print(f"Final answer: {result}")
print()


# What @double_result actually does:
print("=" * 50)
print("WHAT'S HAPPENING:")
print("=" * 50)
print("""
1. You write:
   @double_result
   def add_doubled(a, b):
       return a + b

2. Python does this behind the scenes:
   add_doubled = double_result(add_doubled)

3. Now add_doubled is actually the wrapper function

4. When you call add_doubled(3, 5):
   - wrapper(3, 5) runs
   - wrapper calls original: func(3, 5) → 8
   - wrapper doubles it: 8 * 2 → 16
   - wrapper returns: 16

VISUAL:
-------
add_doubled(3, 5)
    ↓
[wrapper function]
    ↓
calls original add_doubled(3, 5) → 8
    ↓
doubles: 8 * 2 → 16
    ↓
returns 16
""")

In [ ]:
"""
Simplest Decorator Example - Just a + b
========================================
This example shows how decorators work using basic arithmetic
"""

# ============================================================================
# THE DECORATOR FUNCTION
# ============================================================================

def double_result(func):
    """
    This is a decorator function that doubles the return value of any function.
    
    A decorator is just a function that:
    1. Takes another function as input
    2. Returns a modified version of that function
    
    Parameters:
    -----------
    func : function
        The original function we want to modify (in this case, 'add')
    """
    
    # This is the wrapper function - it replaces the original function
    # When you call add(3, 5), you're actually calling THIS function
    def wrapper(a, b):
        """
        The wrapper intercepts calls to the original function.
        It can do things before calling the original, after calling it, or both.
        
        Parameters:
        -----------
        a, b : numbers
            These are the arguments passed when calling add(a, b)
        """
        
        # STEP 1: Call the original function with the arguments
        # func is the original 'add' function that was passed to double_result
        # This computes a + b and stores it in 'result'
        result = func(a, b)
        
        # STEP 2: Modify the result (this is what the decorator does!)
        # Take whatever the original function returned and double it
        # If result is 8, this returns 16
        return result * 2
    
    # STEP 3: Return the wrapper function (NOT calling it, just returning it)
    # This wrapper will replace the original 'add' function
    # Python will bind the name 'add' to this wrapper instead of the original
    return wrapper


# ============================================================================
# USING THE DECORATOR
# ============================================================================

# The @double_result syntax is Python's decorator syntax
# This line tells Python: "wrap the add function with double_result"
#
# What Python does behind the scenes:
# 1. First, define add normally: def add(a, b): return a + b
# 2. Then immediately do: add = double_result(add)
# 3. Now 'add' refers to the wrapper function, not the original
@double_result
def add(a, b):
    """
    This is the original function - it just adds two numbers.
    Simple arithmetic: a + b
    
    But because of @double_result above it:
    - This function gets passed to double_result()
    - double_result() wraps it in the wrapper function
    - The name 'add' now points to that wrapper
    """
    return a + b


# ============================================================================
# EXECUTION FLOW
# ============================================================================

# Test it
# When this line runs, here's what happens step-by-step:
#
# 1. You call: add(3, 5)
# 2. But 'add' is now the wrapper function (thanks to @double_result)
# 3. wrapper(3, 5) executes:
#    - Calls the original add: func(3, 5)
#    - Original add returns: 3 + 5 = 8
#    - wrapper doubles it: 8 * 2 = 16
#    - wrapper returns: 16
# 4. print() receives: 16
print(add(3, 5))  # Output: 16 (because (3+5)*2 = 16)


# ============================================================================
# VISUAL DIAGRAM OF WHAT'S HAPPENING
# ============================================================================

print("\n" + "=" * 60)
print("VISUAL BREAKDOWN:")
print("=" * 60)
print("""
BEFORE DECORATOR:
-----------------
add(3, 5) ──────> return 3 + 5 ──────> 8


AFTER @double_result DECORATOR:
--------------------------------
add(3, 5)
    │
    ↓
[wrapper function activated]
    │
    ├─> calls original add(3, 5)
    │       │
    │       ↓
    │   returns 8
    │       │
    ├───────┘
    │
    ├─> doubles: 8 * 2
    │
    ↓
returns 16


STRUCTURE IN MEMORY:
--------------------
Name 'add' ──────> wrapper function
                      │
                      └──> has access to original add function
                           (stored as 'func' in wrapper's closure)


KEY INSIGHT:
------------
The original add function still exists in memory!
It's just hidden inside the wrapper function.
The wrapper can call it whenever it wants (which it does),
but it can also do extra stuff (like doubling the result).
""")

In [ ]:
"""
Decorator Example - Subtraction (a - b) with Type Hints
"""

from typing import Callable

# The decorator - makes the result negative
def make_negative(func: Callable[[int, int], int]) -> Callable[[int, int], int]:
    """
    Decorator that converts any result to its negative value
    Example: if function returns 5, decorator makes it -5
    
    Parameters:
    -----------
    func : Callable[[int, int], int]
        A function that takes two integers and returns an integer
        
    Returns:
    --------
    Callable[[int, int], int]
        A wrapper function with the same signature
    """
    def wrapper(a: int, b: int) -> int:
        """
        Wrapper function that intercepts calls to the original function
        
        Parameters:
        -----------
        a : int
            First number (minuend)
        b : int
            Second number (subtrahend)
            
        Returns:
        --------
        int
            The negated result of the original function
        """
        # Call the original function (subtract in this case)
        result: int = func(a, b)
        
        # Convert to negative
        return -result
    
    return wrapper


# Apply decorator to subtraction function
@make_negative
def subtract(a: int, b: int) -> int:
    """
    Subtract b from a
    But @make_negative will flip the sign of the result
    
    Parameters:
    -----------
    a : int
        The number to subtract from (minuend)
    b : int
        The number to subtract (subtrahend)
        
    Returns:
    --------
    int
        The negated difference (-(a - b))
    """
    return a - b


# Test it
print(subtract(10, 3))  # Normal: 10-3=7, With decorator: -7
print(subtract(5, 2))   # Normal: 5-2=3, With decorator: -3


# ============================================================================
# TYPE HINT EXPLANATION
# ============================================================================

print("\n" + "=" * 60)
print("TYPE HINTS BREAKDOWN:")
print("=" * 60)
print("""
Callable[[int, int], int]
    │      │    │     │
    │      │    │     └─> Return type (int)
    │      │    └──────> Second parameter type (int)
    │      └───────────> First parameter type (int)
    └──────────────────> This is a callable (function)

The decorator type signature:
------------------------------
def make_negative(func: Callable[[int, int], int]) 
                       -> Callable[[int, int], int]:
                  │                    │
                  │                    └─> Returns a function
                  └──────────────────────> Takes a function

This means:
- Input: A function that takes (int, int) and returns int
- Output: A function that takes (int, int) and returns int
- Same signature in and out!
""")

-7
-3


### How @ Syntax Works

In [ ]:
# Using @ is equivalent to:
def fibonacci_manual(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci_manual(n - 2) + fibonacci_manual(n - 1))

fibonacci_manual = trace(fibonacci_manual)

print("Manual decoration:")
fibonacci_manual(3)

### The Problem: Lost Metadata

In [ ]:
# Decorated function loses its identity
print(f"Function name: {fibonacci}")
print(f"Function __name__: {fibonacci.__name__}")

In [ ]:
# help() shows wrapper, not fibonacci!
help(fibonacci)

In [ ]:
# Pickle fails
import pickle

try:
    pickle.dumps(fibonacci)
except AttributeError as e:
    print(f"AttributeError: {e}")

### Solution: functools.wraps

In [ ]:
from functools import wraps

# GOOD: Use @wraps to preserve metadata
def trace_better(func):
    @wraps(func)  # This is the key!
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        return result
    return wrapper

@trace_better
def fibonacci_fixed(n):# -> Any | int:
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci_fixed(n - 2) + fibonacci_fixed(n - 1))

# Now metadata is preserved!
print(f"Function __name__: {fibonacci_fixed.__name__}")

In [ ]:
# help() now shows the correct docstring
help(fibonacci_fixed)

In [ ]:
# Pickle works now
pickled = pickle.dumps(fibonacci_fixed)
print(f"✓ Successfully pickled: {len(pickled)} bytes")

### What @wraps Preserves

`@wraps` copies these attributes from the wrapped function:
- `__name__` - Function name
- `__module__` - Module name
- `__doc__` - Docstring
- `__annotations__` - Type annotations
- `__qualname__` - Qualified name
- `__dict__` - Instance dictionary

In [ ]:
# Demonstrate preserved attributes
print(f"Name: {fibonacci_fixed.__name__}")
print(f"Module: {fibonacci_fixed.__module__}")
print(f"Doc: {fibonacci_fixed.__doc__}")
print(f"Qualname: {fibonacci_fixed.__qualname__}")

### Real-World Decorator Example

In [ ]:
import time

def timing_decorator(func):
    """Decorator that measures function execution time"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'{func.__name__} took {end - start:.4f} seconds')
        return result
    return wrapper

@timing_decorator
def slow_function(n):
    """Simulate slow computation"""
    time.sleep(0.1)
    return sum(range(n))

result = slow_function(1000)
print(f'Result: {result}')

### Decorator Best Practices

✓ **Always use `@wraps`** when defining decorators

✓ **Use `*args` and `**kwargs`** to accept any arguments

✓ **Return the result** from the wrapped function

✓ **Document behavior** in decorator docstring

---

## Chapter 3 Summary

### Key Takeaways

| Item | Key Principle |
|------|---------------|
| **19** | Never unpack more than 3 return values; use namedtuple or classes |
| **20** | Raise exceptions instead of returning None for errors |
| **21** | Use `nonlocal` or classes for closures that modify enclosing scope |
| **22** | Use `*args` for optional positional arguments (with caution) |
| **23** | Provide optional behavior with keyword arguments |
| **24** | Use `None` and docstrings for dynamic default arguments |
| **25** | Use `/` for positional-only and `*` for keyword-only arguments |
| **26** | Always use `functools.wraps` when writing decorators |

### Best Practices Checklist

- ✓ Return simple tuples (≤3 values) or use namedtuple
- ✓ Raise exceptions for special cases
- ✓ Use type annotations for clarity
- ✓ Leverage keyword arguments for optional behavior
- ✓ Avoid mutable default arguments
- ✓ Enforce argument passing style with `/` and `*`
- ✓ Always use `@wraps` in decorators